<a href="https://colab.research.google.com/github/vaikunthcreates/TimeSeries/blob/main/Time_Series_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import necessary libraries
!pip install pymannkendall
import pandas as pd
import numpy as np
import datetime
import statsmodels.api as sm
import tensorflow as tf
import pymannkendall as mk
import math
import warnings
warnings.filterwarnings("ignore")
import itertools 
from fbprophet import Prophet
from statsmodels.tsa.stattools import acf  
from statsmodels.tsa.stattools import pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.layers.core import Dense, Activation, Dropout
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from xgboost import plot_importance, plot_tree
from sklearn.metrics import mean_squared_error, mean_absolute_error
import io
from google.colab import files


In [ ]:
#Upload file
uploaded = files.upload()
df= pd.read_excel(io.BytesIO(uploaded['sales_mock_kaggle.xlsx']))
df = df.rename(columns={'Sale':'Target'})
df.set_index('Date',inplace=True) 
#Train-Test Split
train,test = train_test_split(df,test_size=0.2,shuffle=False)

Saving sales_mock_kaggle.xlsx to sales_mock_kaggle.xlsx


In [ ]:
#Read file
df = pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv')
df = df.rename(columns={'Temp':'Target'})
df['Date'] =  pd.to_datetime(df['Date'], infer_datetime_format=True)
df.set_index('Date',inplace=True) 
#Train-Test Split
train,test = train_test_split(df,test_size=0.2,shuffle=False)
test_or=test
df.head(20)

,Target
Date,
1981-01-01,20.7
1981-01-02,17.9
1981-01-03,18.8
1981-01-04,14.6
1981-01-05,15.8
1981-01-06,15.8
1981-01-07,15.8
1981-01-08,17.4
1981-01-09,21.8


**Creating gateways/checkpoints:**

In [ ]:
#Length of Data
if len(df)<1000:
  length='small'
  print('This data has less than 1000 records and is limited')
else:
  length='big'
  print('This data has over 1000 records')
    
#Stationarity Test

adf = sm.tsa.stattools.adfuller(df.Target) 

if adf[1] < 0.05:
  stationary='true'
  print('Time Series is Stationary')
else:
  stationary='false'
  print('Time Series is not Stationary')

#Skewness Test
from scipy.stats import skew
if -0.5<float(skew(df.Target))<0.5:
  skewness = 'insignificant'
  print("Data is fairly symmetrical")
elif -1<float(skew(df))<0.5 or 0.5<float(skew(df))<1:
  skewness='moderate'
  print("Data is moderately skewed")
else:
  skewness = 'high'
  print("Data is highly skewed")

#Trend Test
mktest=mk.seasonal_test(df.Target)
if mktest[3] < 0.05:
  trend = 'significant'
  print('Time Series displays significant trend.')
else:
  trend = 'insignificant'
  print('Time Series does not display significant trend.')

#Seasonality Test
res = sm.tsa.ARMA(df.Target, (1,1)).fit(disp=-1)
jungbox = sm.stats.acorr_ljungbox(res.resid, lags=[30])
jungbox = round(np.float(jungbox[1]))
if jungbox < 0.05:
  seasonality = 'significant'
  print('Time Series displays significant seasonality.')
else:
  seasonality = 'insignificant'
  print('Time Series does not display significant seasonality.')

#Null Values
null_values=df.Target.isnull().sum()
null_values=int(null_values)*100/len(df.Target)
if null_values<5:
  null_check = 'insignificant' 
  print('Null values are insignificant')
else:
  null_check = 'significant' 
  print('Null values are significant')



This data has over 1000 records
Time Series is Stationary
Data is fairly symmetrical
Time Series does not display significant trend.
Time Series displays significant seasonality.
Null values are insignificant


In [ ]:
#XGBOOST model
def create_features(df, label=None):
    """
    Creates time series features from datetime index
    """
    df['date'] = df.index
    df['dayofweek'] = df['date'].dt.dayofweek
    df['quarter'] = df['date'].dt.quarter
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['dayofyear'] = df['date'].dt.dayofyear
    df['dayofmonth'] = df['date'].dt.day
    df['weekofyear'] = df['date'].dt.weekofyear
    
    X = df[['dayofweek','quarter','month','year',
           'dayofyear','dayofmonth','weekofyear']]
    if label:
        y = df[label]
        return X, y
    return X
train,test = train_test_split(df,test_size=0.2,shuffle=False)
X_train, y_train = create_features(train, label='Target')
X_test, y_test = create_features(test, label='Target')
reg = xgb.XGBRegressor(n_estimators=1000)
reg.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        early_stopping_rounds=50,
       verbose=False)
xgboost = reg.predict(X_test)
xgboost = pd.DataFrame(xgboost, columns=['xgboost'])
xgboost.index = test.index
#result.to_csv('XGBoost.csv')
#files.download('XGBoost.csv')

[11:14:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
#PMDARIMA
#model = pm.auto_arima(df.Target,m=12,seasonal=True,start_p=0,start_q=0,test='adf',error_action='ignore',suppress_warnings=True,stepwise=True,trace=True)
#model.summary()
#model.fit(train.Target)
#forecast = model.predict(n_periods=len(test))
#forecast = pd.DataFrame(forecast)
#forecast.to_csv('PMDARIMA.csv')
#files.download('PMDARIMA.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#SARIMAX
"""
Finding optimal parameters
"""
p = range(0, 3)
d = range(0,2)
q = range(0, 3)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]
best_result=[0,0,1000000]
for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(train.Target,
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)
            results = mod.fit()
            print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))
            if results.aic<best_result[2]:
              best_result = [param,param_seasonal,results.aic]
        except:
            continue
print('\nBest Result:', best_result)

#Build model, fit and predict
mod = sm.tsa.statespace.SARIMAX(train.Target, order=(best_result[0][0], best_result[0][1], best_result[0][1]),
                                seasonal_order=(best_result[1][0], best_result[1][1], best_result[1][2], best_result[1][3]),
                                enforce_stationarity=False, enforce_invertibility=False)
results = mod.fit()
print(results.summary())
sarimax = results.predict(start = len(train), end = len(df) - 1, type = 'levels')
sarimax = pd.DataFrame(sarimax, columns=['sarimax'])
sarimax.index = test.index
#result.to_csv('SARIMAX.csv')
#files.download('SARIMAX.csv')

ARIMA(0, 0, 0)x(0, 0, 0, 12)12 - AIC:50931.07346054893
ARIMA(0, 0, 0)x(0, 0, 1, 12)12 - AIC:48808.75339076919
ARIMA(0, 0, 0)x(0, 0, 2, 12)12 - AIC:44021.56271869452
ARIMA(0, 0, 0)x(0, 1, 0, 12)12 - AIC:32895.073309755426
ARIMA(0, 0, 0)x(0, 1, 1, 12)12 - AIC:32730.788746282124
ARIMA(0, 0, 0)x(0, 1, 2, 12)12 - AIC:32591.577713134833
ARIMA(0, 0, 0)x(1, 0, 0, 12)12 - AIC:32815.440821679935
ARIMA(0, 0, 0)x(1, 0, 1, 12)12 - AIC:32757.520211478848
ARIMA(0, 0, 0)x(1, 0, 2, 12)12 - AIC:32596.597896081104
ARIMA(0, 0, 0)x(1, 1, 0, 12)12 - AIC:32742.998774285195
ARIMA(0, 0, 0)x(1, 1, 1, 12)12 - AIC:32732.719370180504
ARIMA(0, 0, 0)x(1, 1, 2, 12)12 - AIC:32587.142695259754
ARIMA(0, 0, 0)x(2, 0, 0, 12)12 - AIC:32621.13542015048
ARIMA(0, 0, 0)x(2, 0, 1, 12)12 - AIC:32608.806413927665
ARIMA(0, 0, 0)x(2, 0, 2, 12)12 - AIC:32597.275806630612
ARIMA(0, 0, 0)x(2, 1, 0, 12)12 - AIC:32602.805911844553
ARIMA(0, 0, 0)x(2, 1, 1, 12)12 - AIC:32594.07028236982
ARIMA(0, 0, 0)x(2, 1, 2, 12)12 - AIC:32570.5100369832

In [ ]:
#LSTM
#Scale data for LSTM
scaler=MinMaxScaler(feature_range=(0,1))
df_scaled=scaler.fit_transform(np.array(df.Target).reshape(-1,1))
features=df_scaled
target=df_scaled[:,0]
#Utility class to produce batches for training/validation 
TimeseriesGenerator(features, target, length=2, 
                    sampling_rate=1, batch_size=1)[0]
x_train, x_test, y_train, y_test = train_test_split(features, 
                                                    target, test_size=0.20, random_state=123, shuffle = False)
win_length=1
batch_size=64
num_features=1
train_generator = TimeseriesGenerator(x_train, y_train, 
                                      length=win_length, sampling_rate=1, batch_size=batch_size)
test_generator = TimeseriesGenerator(x_test, y_test, 
                                     length=win_length, sampling_rate=1, batch_size=batch_size)
#Build model, fit and predict
model=Sequential()
model.add(LSTM(50,return_sequences=True,input_shape = (win_length, num_features)))
model.add(Dropout(0.2))
model.add(LSTM(50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(loss=tf.losses.MeanSquaredError(),
              optimizer=tf.optimizers.Adam(),metrics=[tf.metrics.MeanAbsoluteError()])
history = model.fit(train_generator, epochs=50, validation_data=test_generator,shuffle=False)
predictions=model.predict(test_generator)
x_test[:,1:][win_length:]
df_pred=pd.concat([pd.DataFrame(predictions), pd.DataFrame(x_test[:,1:][win_length:])],axis=1)
#Inverse transform the scaled data
rev_trans=scaler.inverse_transform(df_pred)
df_final=df[predictions.shape[0]*-1:]
df_final['lstm']=rev_trans[:,0]
lstm=df_final.drop(['Target'], axis=1)

Epoch 1/50
46/46 [==============================] - 7s 51ms/step - loss: 0.1803 - mean_absolute_error: 0.3838 - val_loss: 0.0234 - val_mean_absolute_error: 0.1232
Epoch 2/50
46/46 [==============================] - 0s 8ms/step - loss: 0.0252 - mean_absolute_error: 0.1269 - val_loss: 0.0163 - val_mean_absolute_error: 0.1042
Epoch 3/50
46/46 [==============================] - 0s 8ms/step - loss: 0.0213 - mean_absolute_error: 0.1166 - val_loss: 0.0144 - val_mean_absolute_error: 0.0980
Epoch 4/50
46/46 [==============================] - 0s 8ms/step - loss: 0.0190 - mean_absolute_error: 0.1094 - val_loss: 0.0128 - val_mean_absolute_error: 0.0921
Epoch 5/50
46/46 [==============================] - 0s 8ms/step - loss: 0.0172 - mean_absolute_error: 0.1035 - val_loss: 0.0112 - val_mean_absolute_error: 0.0858
Epoch 6/50
46/46 [==============================] - 0s 8ms/step - loss: 0.0154 - mean_absolute_error: 0.0985 - val_loss: 0.0101 - val_mean_absolute_error: 0.0813
Epoch 7/50
46/46 [=========

In [ ]:
#FB Prophet Model
m=Prophet()
df1=df
#Transform date column as 'ds' and target as 'y'
df1['ds'] =  df1.index
df1 = df1.rename(columns={'Target':'y'})
train_fb,test_fb = train_test_split(df1,test_size=0.2,shuffle=False)
#Fit model and predict
m.fit(train_fb)
prediction_dates = m.make_future_dataframe(periods=len(test_fb)+1)
prediction = m.predict(prediction_dates)
prediction.set_index('ds',inplace=True)
fbprophet = prediction['yhat']
fbprophet=fbprophet.loc[test_fb.index[0]:test_fb.index[-1],]
fbprophet=pd.DataFrame(fbprophet)
fbprophet = fbprophet.rename(columns={'yhat':'fbprophet'})
#fbprophet.to_csv('FBProphet.csv')
#files.download('FBProphet.csv')

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [ ]:
#Creating dataframe with all model predictions and calculating error value
test_or['xgboost']= xgboost
test_or['sarimax']= sarimax
test_or['lstm']= lstm
test_or['fbprophet']= fbprophet
columns = {'xgboost','sarimax','lstm','fbprophet'}
for i in columns:
  free=pd.notnull(test_or[i])
  test_or[free]
  #if test_or[i].isnull()
  print('RMSE of', i , mean_squared_error(test_or[free].Target,
                                          test_or[free][i],squared=False))


RMSE of sarimax 5.267037715183695
RMSE of fbprophet 2.9237027466756524
RMSE of xgboost 2.6005096907243765
RMSE of lstm 2.383544190647838


In [ ]:
#Ensemble Prediction
if length == 'small' and trend == 'insignificant': 
  forecast = test_or.sarimax*2/5+test_or.xgboost*3/5
  print(forecast)
elif length == 'big' and trend == 'insignificant' and seasonality == 'significant' :
  forecast = (test_or.xgboost+test_or.fbprophet+test_or.lstm)/3
  print(forecast)
elif length=='big' and trend=='significant' and seasonality =='insignificant':
  forecast == test_or.sarimax*3/5 +test_or.lstm*2/5
  print(forecast)
else:
  forecast = (test_or.xgboost + test_or.sarimax + test_or.fbprophet + test_or.lstm)/5

Date
1989-01-01          NaN
1989-01-02    15.098211
1989-01-03    15.902129
1989-01-04    16.182786
1989-01-05    15.737969
                ...    
1990-12-27    14.787174
1990-12-28    14.677146
1990-12-29    14.589954
1990-12-30    14.858042
1990-12-31    15.187118
Length: 730, dtype: float64
